In [75]:
import nltk
from pymongo import MongoClient

In [76]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.conan

In [6]:
test_doc = db.casting_pool.find_one({})['wiki_doc']

In [7]:
test_doc

' from wikipedia, the free encyclopedia jump to navigation jump to search irish actor peter o toolepeter o toole as t. e. lawrence in lawrence of arabia ( )bornpeter seamus o toole( ) august disputed either connemara, county galway, ireland or leeds, yorkshire, englanddied december ( ) (aged )london, englandnationalitybritishalma materroyal academy of dramatic artoccupationactor, authoryears active – height ft in ( cm)spouse(s)siân phillips(m. ; div. )partner(s)karen brown ( )childrenkate o toole, patricia o toole, and lorcan o toole peter seamus o toole ( oʊˈtuːl ; august – december ) was a british stage and film actor of irish descent. he attended the royal academy of dramatic art and began working in the theatre, gaining recognition as a shakespearean actor at the bristol old vic and with the english stage company before making his film debut in . he achieved international recognition playing t. e. lawrence in lawrence of arabia ( ) for which he received his first nomination for the

In [77]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [11]:
cv = CountVectorizer(stop_words='english')

In [13]:
X = cv.fit_transform([test_doc])

In [15]:
pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

,aa,abbey,able,abraham,academy,accepted,accepting,accessed,acclaim,acclaimed,...,youtube,yul,yves,yvonne,zealand,zimbalist,zorba,zukor,zulu,áras
0,2,4,1,1,23,1,1,2,1,1,...,1,1,1,1,1,1,1,1,1,1


In [33]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [34]:
split_doc = test_doc.split()
wrds = [stemmer.stem(wrd) for wrd in split_doc]

In [35]:
X = cv.fit_transform([' '.join(wrds)])
pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

,aa,abbey,abl,abraham,academi,accept,accepting,access,acclaim,achiev,...,youtube,yul,yves,yvonn,zealand,zimbalist,zorba,zukor,zulu,áras
0,2,4,1,1,23,1,1,2,2,1,...,1,1,1,1,1,1,1,1,1,1


In [78]:
casting_pool = db.casting_pool

In [101]:
corpus = []

import pprint
for person in casting_pool.find():
    corpus.append(person)

In [112]:
pprint.pprint(corpus[4]['stemmed_wiki_doc'])

('from wikipedia, the free encyclopedia jump to navig jump to search for other '
 'peopl name richard burton, see richard burton (disambiguation). welsh actor '
 'richard burtoncbeburton in the robe ( )bornrichard walter jenkin jr.( ) '
 'novemb pontrhydyfen, glamorgan, walesdi august ( ) (age )céligny, geneva, '
 'switzerlandoccupationactoryear activ – spouse(s)sybil williams(m. ; div. '
 ')elizabeth taylor(m. ; div. )(m. ; div. )suzi miller(m. ; div. )salli hay '
 '(m. )children , includ kate burton richard burton, cbe ( ˈbɜːrtən ; born '
 'richard walter jenkin jr.; novemb – august ) wa a welsh actor. note for hi '
 'melliflu bariton voice, burton establish himself as a formid shakespearean '
 'actor in the s, and he gave a memor perform of hamlet in . he wa call the '
 'natur successor to olivi by critic and dramaturg kenneth tynan. an '
 'alcoholic, burton s failur to live up to those expect disappoint critic and '
 'colleagu and fuell hi legend as a great thespian wastrel. burton

In [109]:
for person in corpus:
    split_doc = person['wiki_doc'].split()
    wrds = [stemmer.stem(wrd) for wrd in split_doc]
    stemmed_doc = ' '.join(wrds)
    person['stemmed_wiki_doc'] = stemmed_doc

In [113]:
for person in corpus:
    casting_pool.find_one_and_replace({'_id': person['_id']}, person)

In [114]:
people = []

for person in casting_pool.find():
    people.append(person)

In [115]:
people[0]

{'_id': ObjectId('5c6dc5f6fc0c4b94ecceeae9'),
 'name': "Peter O'Toole",
 'url': 'https://en.wikipedia.org/wiki/Peter_O%27Toole',
 'wiki_doc': ' from wikipedia, the free encyclopedia jump to navigation jump to search irish actor peter o toolepeter o toole as t. e. lawrence in lawrence of arabia ( )bornpeter seamus o toole( ) august disputed either connemara, county galway, ireland or leeds, yorkshire, englanddied december ( ) (aged )london, englandnationalitybritishalma materroyal academy of dramatic artoccupationactor, authoryears active – height ft in ( cm)spouse(s)siân phillips(m. ; div. )partner(s)karen brown ( )childrenkate o toole, patricia o toole, and lorcan o toole peter seamus o toole ( oʊˈtuːl ; august – december ) was a british stage and film actor of irish descent. he attended the royal academy of dramatic art and began working in the theatre, gaining recognition as a shakespearean actor at the bristol old vic and with the english stage company before making his film debut 

In [80]:
len(people)

1060

In [126]:
people[75]

{'_id': ObjectId('5c6dc6eafc0c4b94ecceeb34'),
 'name': 'Al Pacino',
 'url': 'https://en.wikipedia.org/wiki/Al_Pacino',
 'wiki_doc': ' from wikipedia, the free encyclopedia this is the latest accepted revision, reviewed on february . jump to navigation jump to search al pacinopacino in bornalfredo james pacino ( ) april , (age )new york city, u.s.alma materactors studiohb studiooccupation actor filmmaker years active –presentpartner(s)jan tarrant ( – )beverly d angelo ( – )children alfredo james pacino ( pəˈtʃiːnoʊ ; born april , ) is an american actor and filmmaker. pacino has had a career spanning more than five decades, during which time he has received numerous accolades and honors both competitive and honorary, among them an academy award, two tony awards, two primetime emmy awards, a british academy film award, four golden globe awards, the lifetime achievement award from the american film institute, the golden globe cecil b. demille award, and the national medal of arts. he is on

In [117]:
docs = [p['stemmed_wiki_doc'] for p in people]
X = cv.fit_transform(docs)
doc_matrix = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())
doc_ids = [p['_id'] for p in people]
idx = pd.Index(data=doc_ids)


In [129]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv_tfidf = TfidfVectorizer(stop_words='english')
X_tfidf = cv_tfidf.fit_transform(docs).toarray()

df_tfidf = pd.DataFrame(X_tfidf, columns=cv_tfidf.get_feature_names(), index=idx)


In [130]:
df_tfidf.head()

,___,___________,__philip,_big_fish,_blige,_castellano,_english_,_gazzo,_grant,_henson,...,空の穴,菊地,裴淳华,裴淳华是什么鬼,남자들,빅뱅,빅뱅家의,승리와의,인터뷰,화보
5c6dc5f6fc0c4b94ecceeae9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5c6dc5f9fc0c4b94ecceeaea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5c6dc5fdfc0c4b94ecceeaeb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5c6dc600fc0c4b94ecceeaec,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5c6dc603fc0c4b94ecceeaed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [136]:
al_pac = casting_pool.find_one({'name': 'Al Pacino'})

In [137]:
jack_nick = casting_pool.find_one({'name': 'Jack Nicholson'})

In [152]:
bill = casting_pool.find_one({'name': 'Bill Clinton'})

In [155]:
import numpy
import math

def cosine_distance(u, v):
    """
    Returns the cosine of the angle between vectors v and u. This is equal to
    u.v / |u||v|.
    """
    return numpy.dot(u, v) / (math.sqrt(numpy.dot(u, u)) * math.sqrt(numpy.dot(v, v)))

cosine_distance(df_tfidf.loc[al_pac['_id']], df_tfidf.loc[jodie_fost['_id']])

cosine_distance(df_tfidf.loc[al_pac['_id']], df_tfidf.loc[jack_nick['_id']])

cosine_distance(df_tfidf.loc[al_pac['_id']], df_tfidf.loc[bill['_id']])


0.10483728390882005

In [156]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(5)
topics = lsa.fit_transform(df_tfidf)
lsa.explained_variance_ratio_

array([0.01373305, 0.05316908, 0.02886831, 0.01768083, 0.01322444])

In [164]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        

display_topics(lsa, cv_tfidf.get_feature_names(), 8)


Topic  0
award, film, retriev, wa, best, actor, actress, output

Topic  1
actress, streep, meryl, jessica, helen, ann, blanchett, salli

Topic  2
retriev, archiv, origin, streep, wa, januari, septemb, octob

Topic  3
streep, nicholson, meryl, hoffman, robert, jack, award, fonda

Topic  4
award, best, actor, archiv, origin, critic, support, film


In [92]:
d1 = len(docs[0])
for d in docs:
    print (len(d)==d1)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [93]:
len(people)

1060

In [100]:
len(people[2]['stemmed_wiki_doc'])

50138